In [ ]:
import tweepy
import json
import time
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Authorization Codes
consumer_key = "ZCVjbgfaSGlrJVh7T2cqIFfMR"
consumer_secret = "yah2rFbZE1qX9n1MDaqObT1PfCOSYlvxIyXYuCO4ZrSUfSvjEC"
access_key = "781555439183859712-tS4fKwo03ujDtNL0UibVIXauXRFdFvs"
access_secret = "3YrATCHVvP2atvFs4M6VJ0dFvsdE3BBKoZ2OcvKdqPzJp"

# OAuth
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)

# API Call
api = tweepy.API(auth, parser = tweepy.parsers.JSONParser())

In [ ]:
# Test API by pulling data from Donald Trumps twitter
data = api.user_timeline(id='realDonaldTrump', count = 200)  #Example:@realDonaldTrump
for i in range(10):
    print([i],data[i]['text'])

In [15]:
search_query =  '"Ethereum" -filter:retweets'

In [16]:
data = api.search(q = search_query, count = 100, lang = 'en', result_type = 'mixed')

In [ ]:
#all_data = list(data[1])

all_data = list(data.values())[1]


In [ ]:
while(len(all_data) <= 30000):
    time.sleep(5)
    last = all_data[-1]['id']
    data = api.search(q = search_query, count = 100, lang = 'en', result_type = 'mixed', since_id = last)
    all_data += list(data.values())[1][1:]

In [69]:
data[1][4]['text']
all_data[1][2]['text']
#for i in range(1000):
#    print([i],all_data[1][i]['text'])


IndexError: string index out of range